<a href="https://colab.research.google.com/github/Rishap100/KmerEBV/blob/main/KmerEBV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/DeepEBV-master

Mounted at /content/drive
/content/drive/MyDrive/DeepEBV-master


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, precision_score, roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
import numpy as np

test_data = np.load('/content/drive/MyDrive/DeepEBV-master/EBVdata/VISDB_Data/VISDB_Test_Data.npy')
test_label = np.load('/content/drive/MyDrive/DeepEBV-master/EBVdata/VISDB_Data/VISDB_Test_Label.npy')

new_test_data = test_data.reshape(test_data.shape[0],test_data.shape[1]*test_data.shape[2])
print(np.shape(new_test_data))

(12144, 8000)


In [ ]:
x_train_data = np.chararray((12144,2000))
for i in range(len(new_test_data)):
  s=""
  k=0
  for j in range(len(new_test_data[i])):
    if(j%4==0 and new_test_data[i][j]==1):
      #s=s+"A"
      x_train_data[i][k] = 'A'
      k=k+1
    elif(j%4==1 and new_test_data[i][j]==1):
      x_train_data[i][k] = 'T'
      k=k+1
    elif(j%4==2 and new_test_data[i][j]==1):
      x_train_data[i][k] = 'C'
      k=k+1
    elif(j%4==3 and new_test_data[i][j]==1):
      x_train_data[i][k] = 'G'
      k=k+1
    else:
      continue
  #x_train_data[i]=s
print(x_train_data[0])

[b'C' b'T' b'T' ... b'T' b'G' b'T']


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, precision_score, roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
import numpy as np
class kmer_featurization:

  def __init__(self, k):
    """
    seqs: a list of DNA sequences
    k: the "k" in k-mer
    """
    self.k = k
    self.letters = ['A', 'T', 'C', 'G']
    self.multiplyBy = 4 ** np.arange(k-1, -1, -1) # the multiplying number for each digit position in the k-number system
    self.n = 4**k # number of possible k-mers

  def obtain_kmer_feature_for_a_list_of_sequences(self, seqs, write_number_of_occurrences=False):
    """
    Given a list of m DNA sequences, return a 2-d array with shape (m, 4**k) for the 1-hot representation of the kmer features.
    Args:
      write_number_of_occurrences:
        a boolean. If False, then in the 1-hot representation, the percentage of the occurrence of a kmer will be recorded; otherwise the number of occurrences will be recorded. Default False.    
    """
    kmer_features = []
    for seq in seqs:
      this_kmer_feature = self.obtain_kmer_feature_for_one_sequence(seq.upper(), write_number_of_occurrences=write_number_of_occurrences)
      kmer_features.append(this_kmer_feature)

    kmer_features = np.array(kmer_features)

    return kmer_features

  def obtain_kmer_feature_for_one_sequence(self, seq, write_number_of_occurrences=False):
    """
    Given a DNA sequence, return the 1-hot representation of its kmer feature.
    Args:
      seq: 
        a string, a DNA sequence
      write_number_of_occurrences:
        a boolean. If False, then in the 1-hot representation, the percentage of the occurrence of a kmer will be recorded; otherwise the number of occurrences will be recorded. Default False.
    """
    number_of_kmers = len(seq) - self.k + 1

    kmer_feature = np.zeros(self.n)

    for i in range(number_of_kmers):
      this_kmer = seq[i:(i+self.k)]
      this_numbering = self.kmer_numbering_for_one_kmer(this_kmer)
      kmer_feature[this_numbering] += 1

    if not write_number_of_occurrences:
      kmer_feature = kmer_feature / number_of_kmers

    return kmer_feature

  def kmer_numbering_for_one_kmer(self, kmer):
    """
    Given a k-mer, return its numbering (the 0-based position in 1-hot representation)
    """
    digits = []
    for letter in kmer:
      digits.append(self.letters.index(letter))

    digits = np.array(digits)

    numbering = (digits * self.multiplyBy).sum()

    return numbering

In [ ]:
seq_list_train = []
for i in range (len(x_train_data)):
  s = x_train_data[i].decode("utf-8")
  string = ""
  for j in range(len(s)):
    string += str(s[j])
  #seq_list[i] = string
  seq_list_train.append(string)
print(len(seq_list_train))

12144


In [ ]:
np.save('EBVdata/seq_list_train',seq_list_train)

In [ ]:
np.shape(seq_list_train)

(12144,)

In [ ]:
seq_list_train[0]

In [ ]:
k = 6  # choose the value for k

obj = kmer_featurization(k)  # initialize a kmer_featurization object
kmer_features = obj.obtain_kmer_feature_for_a_list_of_sequences(seq_list_train, write_number_of_occurrences=False)

In [ ]:
np.save('EBVdata/kmer_features_train5', kmer_features)

In [ ]:
kmer_features = np.load('/content/drive/MyDrive/DeepEBV-master/EBVdata/kmer_features_train5.npy')

NameError: ignored

In [ ]:
np.shape(kmer_features)

(12144, 4096)

In [ ]:
kmer_features = np.load('/content/drive/MyDrive/DeepEBV-master/EBVdata/kmer_features_train.npy')

In [ ]:
np.shape(kmer_features)

(29293, 4096)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(kmer_features, test_label, test_size = 0.20)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
model = Sequential()
model.add(Dense(4096,input_dim = 4096, activation = 'relu'))
model.add(Dense(2000,activation= 'relu'))
model.add(Dense(1000,activation= 'relu'))
model.add(Dense(500,activation= 'relu'))
model.add(Dense(200,activation= 'relu'))
model.add(Dense(200,activation='relu'))
model.add(Dense(100,activation='relu'))
model.add(Dense(30,activation='relu'))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss = 'binary_crossentropy',optimizer='adam', metrics =['accuracy']) 

In [ ]:
model.fit(X_train, y_train, epochs=10, batch_size= 5,  verbose=2)
model.save("model5.h5")

In [ ]:
from keras.models import load_model
loaded_model = load_model("model5.h5")

In [ ]:
loaded_model.fit(X_train, y_train, epochs=20, batch_size= 5,  verbose=2)
loaded_model.save("model5.h5")

In [ ]:
output = loaded_model.predict(X_test)

In [ ]:
print(output)

[[1.0000000e+00]
 [1.0889024e-02]
 [1.4411700e-12]
 ...
 [3.7081242e-03]
 [1.3716519e-03]
 [6.4550933e-20]]


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
rounded = [int(round(x[0])) for x in output]
bound = np.arange(.5,.7,.03) 
for i in range(len(bound)):
  rounded = [1 if x>bound[i] else 0 for x in output ]

  print("\n Bound : " + str(bound[i]))
  print(confusion_matrix(y_test,rounded))
  print(classification_report(y_test,rounded))


 Bound : 0.5
[[2210    8]
 [  47  164]]
              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99      2218
         1.0       0.95      0.78      0.86       211

    accuracy                           0.98      2429
   macro avg       0.97      0.89      0.92      2429
weighted avg       0.98      0.98      0.98      2429


 Bound : 0.53
[[2210    8]
 [  48  163]]
              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99      2218
         1.0       0.95      0.77      0.85       211

    accuracy                           0.98      2429
   macro avg       0.97      0.88      0.92      2429
weighted avg       0.98      0.98      0.98      2429


 Bound : 0.56
[[2210    8]
 [  48  163]]
              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99      2218
         1.0       0.95      0.77      0.85       211

    accuracy                           0.98      2429
   

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot

# calculate scores
ns_auc = roc_auc_score(y_test, output)

# summarize scores
print(' AUROC = %.3f' % (ns_auc))

 AUROC = 0.923


In [ ]:
cm1 = confusion_matrix(rounded,y_test)
print(cm1)
sensitivity1 = cm1[0,0]/(cm1[0,0]+cm1[1,0])
print('Sensitivity : ', sensitivity1 )

specificity1 = cm1[1,1]/(cm1[0,1]+cm1[1,1])
print('Specificity : ', specificity1)

ppv = cm1[0,0]/(cm1[0,1]+cm1[0,0])
print('PPV',ppv)

npv = cm1[1,1]/(cm1[1,0]+cm1[1,1])
print('NPV', npv)

auc = roc_auc_score(y_test,rounded)
print('AUC',auc)

[[2211   48]
 [   7  163]]
Sensitivity :  0.996844003606853
Specificity :  0.7725118483412322
PPV 0.9787516600265604
NPV 0.9588235294117647
AUC 0.8846779259740425
